<a href="https://colab.research.google.com/github/mananuppadhyay/QNA-Mistral-Langchain/blob/main/RAG_using_Mistral_and_Langchain_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

How to run the notebook:
1. Add the path to the pdf file after uploading it to the colab environment
2. Run all the cells, and the last block has the option for user input query and answer for the same.


Some details:
LLM used: Mistral 7B (sharded version)
Langchain
Vector Database: QDrant

By: Manan Uppadhyay

Give the path to your pdf file

In [ ]:
pdf_file="/content/Assignment 2_Group 5.pdf"

Install dependencies

---



In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
#!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install qdrant_client
!pip install sentence-transformers
!pip install PyPDF2

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from qdrant_client import QdrantClient
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import PyPDF2
from langchain.schema.document import Document


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Loading the Mistral 7B model and creating the pipeline for the task

In [ ]:
model_id = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=600,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

A sample template, question and context to get a response

In [ ]:
template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""
question_p = """Which companies announced their mergers"""
context_p = """ In a landmark fusion of tech titans, Cybervine and QuantumNet announced their merger today, promising to redefine the digital frontier with their combined innovation powerhouse, now known as CyberQuantum."""
prompt = PromptTemplate(template=template, input_variables=["question","context"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"question":question_p,"context":context_p})
response


'Cybervine and QuantumNet announced their merger and formed CyberQuantum.'

The code to get the pdf file, chunk it and store it in a vector DB (Qdrant) to retrieve it while answering the queries

In [ ]:
def get_pdf_content(file_path):
    pdf_file_obj = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
    num_pages = len(pdf_reader.pages)
    page_content = ""
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        page_content += page_obj.extract_text()
    pdf_file_obj.close()
    return page_content

pdf_content = get_pdf_content(pdf_file)

documents = [Document(page_content=pdf_content, metadata={"source": "local"})]


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


#Use Qdrant as the vector storage
vectordb = Qdrant.from_documents(documents=all_splits, embedding=embeddings, location=":memory:",
    prefer_grpc=True,
    collection_name="my_documents",
)

retriever = vectordb.as_retriever()


#Retrieval using LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)
def run_my_rag(qa):
    query = input("Please enter your query: ")
    print(f"Query: {query}\n")
    result = qa.run(query)
    print("\nResult: ", result)




Enter your query after running this cell:

In [ ]:
run_my_rag(qa)

Please enter your query: What is transnational crime?
Query: What is transnational crime?



> Entering new RetrievalQA chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.

Result:   Transnational crime refers to crimes that originated in one or multiple states and impact multiple states. It could involve drug trafficking, human trafficking, smuggling of migrants, illicit trading in firearms, trafficking in natural resources, the illegal trade in wildlife, the sale of fraudulent medicines, or cybercrime. Transnational crime is influenced by cultural and technological globalization and can involve local gangs and organizations that have far-reaching impacts or the creation of large-
